# High Resolution Conflict Forecasting with Spatial Convolutions and Long Short-Term Memory

## Replication Archive

[Benjamin J. Radford](https://www.benradford.com)    
Assistant Professor  
UNC Charlotte  
bradfor7@uncc.edu  

This file produces, fits, and saves the Single Feature Model.

## Imports and seeds

In [1]:
import sys
import os
import gc
import logging

import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load

from itertools import product
from math import isnan

import views
from views import Period, Model, Downsampling
from views.utils.data import assign_into_df
from views.apps.transforms import lib as translib
from views.apps.evaluation import lib as evallib, feature_importance as fi
from views.apps.model import api
from views.apps.extras import extras

import keras
from keras.models import Model
from keras.layers import Input, ConvLSTM2D, Activation, Conv3D, BatchNormalization, Dropout, Bidirectional, GaussianNoise
from keras import optimizers

import tensorflow as tf

import random
import geoplot as gplt
import contextily as ctx

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1 import make_axes_locatable

from numpy.random import seed
seed(1234)
tf.random.set_seed(1234)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Prepare data

In [2]:
# Do you wish to fetch the latest public data? If so, change False to True and run this cell
# Cells below will fail if this is not run if you haven't imported data yourself yet.
redownload_data = False

if redownload_data:
    path_zip = views.apps.data.public.fetch_latest_zip_from_website(path_dir_destination=views.DIR_SCRATCH)
    views.apps.data.public.import_tables_and_geoms(tables=views.TABLES, geometries=views.GEOMETRIES, path_zip=path_zip)

dataset = views.DATASETS["pgm_africa_imp_0"]
df = dataset.gdf
df.reset_index(inplace=True)

update = pd.read_csv("../../data/pgm.csv")
df = pd.merge(df[["geom","pg_id","month_id"]], update, on=["pg_id","month_id"])

In [3]:
df = df.loc[(df["year"]<2021) & (df["year"]>1989)]
df = df.loc[(df["year"]<2020) | (df["month"]<9)]
df["coordx"] = df["geom"].apply(lambda x: x.centroid.x)
df["coordy"] = df["geom"].apply(lambda y: y.centroid.y)
df["col_idx"] = [int(a) for a in list((df["coordx"] - df["coordx"].min())*2)]
df["row_idx"] = [int(a) for a in list((df["coordy"] - df["coordy"].min())*2)]
df["year_idx"] = [int(a) for a in list((df["year"] - df["year"].min()))]
df["month_idx"] = [int(a) for a in list((df["month"] - df["month"].min()))]
df["year_month_idx"] = [int(a) for a in list((df["month_id"] - df["month_id"].min()))]

df.drop("geom", inplace=True, axis=1)


##
## Make Lags
##
df1 = df[["year_month_idx","pg_id","ln_ged_best_sb"]].copy()
df2 = df[["year_month_idx","pg_id","ln_ged_best_sb"]].copy()
df3 = df[["year_month_idx","pg_id","ln_ged_best_sb"]].copy()
df4 = df[["year_month_idx","pg_id","ln_ged_best_sb"]].copy()
df5 = df[["year_month_idx","pg_id","ln_ged_best_sb"]].copy()
df6 = df[["year_month_idx","pg_id","ln_ged_best_sb"]].copy()
df7 = df[["year_month_idx","pg_id","ln_ged_best_sb"]].copy()

df1["year_month_idx"] = df1["year_month_idx"]+1
df2["year_month_idx"] = df2["year_month_idx"]+2
df3["year_month_idx"] = df3["year_month_idx"]+3
df4["year_month_idx"] = df4["year_month_idx"]+4
df5["year_month_idx"] = df5["year_month_idx"]+5
df6["year_month_idx"] = df6["year_month_idx"]+6
df7["year_month_idx"] = df7["year_month_idx"]+7

df1.columns = ["year_month_idx","pg_id","ln_ged_best_sb_l1"]
df2.columns = ["year_month_idx","pg_id","ln_ged_best_sb_l2"]
df3.columns = ["year_month_idx","pg_id","ln_ged_best_sb_l3"]
df4.columns = ["year_month_idx","pg_id","ln_ged_best_sb_l4"]
df5.columns = ["year_month_idx","pg_id","ln_ged_best_sb_l5"]
df6.columns = ["year_month_idx","pg_id","ln_ged_best_sb_l6"]
df7.columns = ["year_month_idx","pg_id","ln_ged_best_sb_l7"]

df = pd.merge(df,df1,how="left",on=["year_month_idx","pg_id"])
df = pd.merge(df,df2,how="left",on=["year_month_idx","pg_id"])
df = pd.merge(df,df3,how="left",on=["year_month_idx","pg_id"])
df = pd.merge(df,df4,how="left",on=["year_month_idx","pg_id"])
df = pd.merge(df,df5,how="left",on=["year_month_idx","pg_id"])
df = pd.merge(df,df6,how="left",on=["year_month_idx","pg_id"])
df = pd.merge(df,df7,how="left",on=["year_month_idx","pg_id"])

df["delta_1"] = df["ln_ged_best_sb"] - df["ln_ged_best_sb_l1"]
df["delta_2"] = df["ln_ged_best_sb"] - df["ln_ged_best_sb_l2"]
df["delta_3"] = df["ln_ged_best_sb"] - df["ln_ged_best_sb_l3"]
df["delta_4"] = df["ln_ged_best_sb"] - df["ln_ged_best_sb_l4"]
df["delta_5"] = df["ln_ged_best_sb"] - df["ln_ged_best_sb_l5"]
df["delta_6"] = df["ln_ged_best_sb"] - df["ln_ged_best_sb_l6"]
df["delta_7"] = df["ln_ged_best_sb"] - df["ln_ged_best_sb_l7"]

del df1
del df2
del df3
del df4
del df5
del df6
del df7

gc.collect()

0

In [4]:
cols_ids = [
    "col_idx",
    "row_idx",
    "pg_id",
    "year",
    "month",
    "year_idx",
    "month_idx",
    "year_month_idx"]

cols_feats = [
    "ln_ged_best_sb",
    "pgd_bdist3",
    "pgd_capdist",
    "pgd_agri_ih",
    "pgd_pop_gpw_sum",
    "pgd_ttime_mean",
    "spdist_pgd_diamsec",
    "pgd_pasture_ih",
    "pgd_savanna_ih",
    "pgd_forest_ih",
    "pgd_urban_ih",
    "pgd_barren_ih",
    "pgd_gcp_mer"
]

cols_lags = [
    "delta_1",
    "delta_2",
    "delta_3",
    "delta_4",
    "delta_5",
    "delta_6",
    "delta_7"
]


In [5]:
subset = df[cols_feats+cols_ids]

##
## Fill in missing grid cells (e.g. water)
## 
all_cells = product(
                list(range(max(subset["year_month_idx"])+1)),
                list(range(max(subset["col_idx"])+1)),
                list(range(max(subset["row_idx"])+1))
                )

all_cells = pd.DataFrame(all_cells,
                         columns=["year_month_idx",
                                  "col_idx",
                                  "row_idx"])

subset = pd.merge(subset, all_cells, how="outer",
                  on=["year_month_idx",
                      "col_idx",
                      "row_idx"])

subset["isnan"] = subset[cols_feats].apply(lambda x: int(any([isnan(a) for a in x])), axis=1)
subset.fillna(0, inplace=True)

X_grouped = subset.groupby(["year_month_idx",
                          "col_idx",
                          "row_idx"])[cols_feats+["isnan"]].mean()
X_grouped.head()

arr = X_grouped.values.reshape((len(X_grouped.index.unique(level=0)),
                              len(X_grouped.index.unique(level=1)),
                              len(X_grouped.index.unique(level=2)),
                              len(cols_feats)+1))

del subset
gc.collect()

print(arr.shape)

(368, 178, 169, 14)


In [6]:
X = arr[:,:,:,:]
Y = arr[:,:,:,0]

Y1 = Y[1:] - Y[0:-1]
Y2 = Y[2:] - Y[0:-2]
Y3 = Y[3:] - Y[0:-3]
Y4 = Y[4:] - Y[0:-4]
Y5 = Y[5:] - Y[0:-5]
Y6 = Y[6:] - Y[0:-6]
Y7 = Y[7:] - Y[0:-7]

filler1 = np.full_like(np.zeros((1,178,169)),np.NaN)
filler2 = np.full_like(np.zeros((2,178,169)),np.NaN)
filler3 = np.full_like(np.zeros((3,178,169)),np.NaN)
filler4 = np.full_like(np.zeros((4,178,169)),np.NaN)
filler5 = np.full_like(np.zeros((5,178,169)),np.NaN)
filler6 = np.full_like(np.zeros((6,178,169)),np.NaN)
filler7 = np.full_like(np.zeros((7,178,169)),np.NaN)

Y1 = np.concatenate((Y1, filler1), axis=0)
Y2 = np.concatenate((Y2, filler2), axis=0)
Y3 = np.concatenate((Y3, filler3), axis=0)
Y4 = np.concatenate((Y4, filler4), axis=0)
Y5 = np.concatenate((Y5, filler5), axis=0)
Y6 = np.concatenate((Y6, filler6), axis=0)
Y7 = np.concatenate((Y7, filler7), axis=0)

YDelta = np.stack((Y1,Y2,Y3,Y4,Y5,Y6,Y7), axis=3)

del Y1
del Y2
del Y3
del Y4
del Y5
del Y6
del Y7
gc.collect()

0

In [7]:
print(X.shape)
print(Y.shape)
print(YDelta.shape)

(368, 178, 169, 14)
(368, 178, 169)
(368, 178, 169, 7)


In [8]:
pred_months = 12

print(X.shape)
print(YDelta.shape)

train_idx = range(pred_months, 12 * 23 + 6)
validate_idx = range(12 * 23 + 6, 12 * 26 + 6)
test_idx = range(12 * 26 + 6, 12 * 30 + 7)

X_train = []
Y_train = []
X_validate = []
Y_validate = []
X_test = []
Y_test = []

for ii in range(X.shape[0]):
    if ii in train_idx:
        X_train.append(X[(ii-pred_months):ii])
        Y_train.append(YDelta[ii-1])
    if ii in validate_idx:
        X_validate.append(X[(ii-pred_months):ii])
        Y_validate.append(YDelta[ii-1])
    if ii in test_idx:
        X_test.append(X[(ii-pred_months):ii])
        Y_test.append(YDelta[ii-1])
        

(368, 178, 169, 14)
(368, 178, 169, 7)


In [9]:
X_train = np.stack(X_train)
Y_train = np.stack(Y_train)
X_validate = np.stack(X_validate)
Y_validate = np.stack(Y_validate)
X_test = np.stack(X_test)
Y_test = np.stack(Y_test)

In [10]:
X_train = X_train[:,:,:,:,0:1]
X_validate = X_validate[:,:,:,:,0:1]
X_test = X_test[:,:,:,:,0:1]

print(X_train.shape)
print(Y_train.shape)
print(X_validate.shape)
print(Y_validate.shape)
print(X_test.shape)
print(Y_test.shape)

(270, 12, 178, 169, 1)
(270, 178, 169, 7)
(36, 12, 178, 169, 1)
(36, 178, 169, 7)
(49, 12, 178, 169, 1)
(49, 178, 169, 7)


In [11]:
in_layer = Input(shape=(None, 178, 169, 1))
norm = BatchNormalization()(in_layer)
conv_layer_3 = Conv3D(filters=20,
                      kernel_size=(1,1,14),
                      activation="tanh",
                      padding="same")(norm)
conv_lstm_1 = Bidirectional(ConvLSTM2D(10,
                         kernel_size = (10,10),
                         activation = "tanh",
                         padding = "same",
                         return_sequences=True))(Dropout(0.2)(conv_layer_3))
conv_lstm_2 = Bidirectional(ConvLSTM2D(5,
                         kernel_size = (5,5),
                         activation = "tanh",
                         padding = "same",
                         return_sequences=True))(Dropout(0.2)(conv_lstm_1))
conv_lstm_3 = ConvLSTM2D(7,
                         kernel_size = (5,5),
                         activation = "linear",
                         padding = "same",
                         return_sequences=False)(Dropout(0.2)(conv_lstm_2))

# output = tf.keras.backend.squeeze(conv_lstm_2, axis=3)


model = Model(inputs=in_layer,
              outputs=conv_lstm_3)
model.compile(optimizer="rmsprop",
                      loss="mean_squared_error")

In [12]:
model.summary(line_length=100)

Model: "model"
____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
input_1 (InputLayer)                         [(None, None, 178, 169, 1)]             0              
____________________________________________________________________________________________________
batch_normalization (BatchNormalization)     (None, None, 178, 169, 1)               4              
____________________________________________________________________________________________________
conv3d (Conv3D)                              (None, None, 178, 169, 20)              300            
____________________________________________________________________________________________________
dropout (Dropout)                            (None, None, 178, 169, 20)              0              
____________________________________________________________________________

## Fit model

Uncomment to fit the model. This is only necessary if you do not want to use the pre-trained model provided in the replication materials.

In [13]:
# model.fit(x=X_train,
#         y=Y_train,
#         batch_size=8,
#         epochs=75,
#         validation_data=(X_validate,Y_validate)
#        )

## Save and load model

Uncomment `model.save(...)` to save your newly trained model. **Warning:** this will overwrite the original model from the paper.

In [14]:
# model.save("../../supplemental_data/single_feature/model_single_feature.h5")
model = keras.models.load_model("../../supplemental_data/single_feature/model_single_feature.h5")

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 178, 169, 1 0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 178, 169, 1) 4         
_________________________________________________________________
conv3d (Conv3D)              (None, None, 178, 169, 20 300       
_________________________________________________________________
dropout (Dropout)            (None, None, 178, 169, 20 0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 178, 169, 20 240080    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 178, 169, 20 0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 178, 169, 10 25040 

## Make all predictions

In [15]:
gc.collect()

all_preds = []

for ii in range(0,X.shape[0]):
    all_preds.append( 
        np.squeeze( 
            model.predict( 
                np.array([X[max(0,ii-pred_months+1):(ii+1),:,:,0:1]])
            )
        )
    )
    
gc.collect()

30081

## Save predictions as Numpy array

In [16]:
all_preds = np.stack(all_preds)
np.save("../../supplemental_data/single_feature/single_feature_predictions.npy", all_preds)
all_preds.shape

(368, 178, 169, 7)